# 実習の説明

## これは何か？

Jupyter notebook というRを使うためのインターフェースの一つです。

- Jupyter notebook の簡単な説明（日本語） https://datumstudio.jp/blog/795

特に、今回は JupyterHub という形で（みなさんの手元のPCではなく）オンラインでJupyter notebookを使う環境を提供しています。
これは、講義などで同一の環境でできるのに便利です。


## 何をするか？

解析デザインの図

- Toi _et al_., Transcriptome Analysis of Individual Stromal Cell Populations Identifies Stroma-Tumor Crosstalk in Mouse Lung Cancer Model, Cell Reports (2015) http://dx.doi.org/10.1016/j.celrep.2015.01.040 


- ソフトウェアやアノテーションデータ http://209.160.41.231/u54/CCCExplorer/
- Gene Expression Omnibus (GEO)のNGSデータのページ https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE59831



![image.png](https://ars.els-cdn.com/content/image/1-s2.0-S2211124715000650-fx1.jpg)
(Image from http://dx.doi.org/10.1016/j.celrep.2015.01.040)

# ここから実習

パッケージ（ライブラリ）を使えるようにするためにロードする

In [85]:
library(readr)
library(dplyr)
library(magrittr)
library(tidyr)
library(ggplot2)


Attaching package: ‘tidyr’

The following object is masked from ‘package:magrittr’:

    extract



### データを読み込む

In [7]:
inputfile = "data/GSE59831_processed_data_FPKM.txt"

df1 = read_tsv(inputfile)

Parsed with column specification:
cols(
  .default = col_double(),
  mice_gene_symbol = col_character(),
  human_gene_symbol = col_character()
)
See spec(...) for full column specifications.


### データの様子を眺める

In [9]:
head(df1)

mice_gene_symbol,human_gene_symbol,Tum1,Tum2,Tum3,WT1,WT2,Tum4,Tum5,WT3,⋯,Tum11,WT8,WT9,WT10,Tum12,Tum13,Tum14,WT11,WT12,WT13
0610007P14Rik,C14orf1,9.31368,10.13680,12.26660,11.92950,11.34170,10.45680,10.44590,13.05840,⋯,21.99580,33.98430,23.50140,24.80460,17.16570,12.51170,16.03630,14.69160,14.03290,14.20440
0610009D07Rik,SF3B14,25.35280,32.25990,26.61000,29.73440,37.16520,37.57330,30.74340,32.30680,⋯,28.76030,21.51540,30.83840,19.26180,35.14690,33.53620,42.73760,29.27860,29.37960,28.06600
0610009O20Rik,KIAA0141,12.45760,10.17690,11.95220,10.62740,7.04763,9.63765,11.11900,13.35890,⋯,20.57930,18.12230,18.65090,18.58170,15.60410,15.34920,13.04400,16.76130,15.91300,18.05510
0610010F05Rik,KIAA1841,5.84091,6.54488,6.65366,4.80074,6.06263,5.57042,5.21223,5.56449,⋯,8.04684,5.92199,6.23551,4.73364,5.98708,4.26902,4.16373,3.47511,3.71875,3.95344
0610010K14Rik,C17orf49,25.63690,24.08560,20.17560,25.34280,29.14040,31.46110,28.30990,22.17650,⋯,18.00200,24.00260,21.85860,23.72630,28.46520,31.19330,26.40880,32.53430,26.93620,29.38080
0610011F06Rik,C16orf13,3.48174,5.79169,3.73809,5.94575,5.98899,9.04125,5.84235,7.36357,⋯,42.61460,38.91730,33.56900,37.34490,20.33750,18.81900,14.17990,14.93870,16.12340,13.39220


In [29]:
dim(df1)

[1] 16024    29

In [26]:
str(df1)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	16024 obs. of  29 variables:
 $ mice_gene_symbol : chr  "0610007P14Rik" "0610009D07Rik" "0610009O20Rik" "0610010F05Rik" ...
 $ human_gene_symbol: chr  "C14orf1" "SF3B14" "KIAA0141" "KIAA1841" ...
 $ Tum1             : num  9.31 25.35 12.46 5.84 25.64 ...
 $ Tum2             : num  10.14 32.26 10.18 6.54 24.09 ...
 $ Tum3             : num  12.27 26.61 11.95 6.65 20.18 ...
 $ WT1              : num  11.9 29.7 10.6 4.8 25.3 ...
 $ WT2              : num  11.34 37.17 7.05 6.06 29.14 ...
 $ Tum4             : num  10.46 37.57 9.64 5.57 31.46 ...
 $ Tum5             : num  10.45 30.74 11.12 5.21 28.31 ...
 $ WT3              : num  13.06 32.31 13.36 5.56 22.18 ...
 $ WT4              : num  8.74 25.08 12.65 3.93 28.72 ...
 $ Tum6             : num  3.9 31.15 3.93 1.56 22.08 ...
 $ Tum7             : num  2.54 33.48 3.74 1.68 19.04 ...
 $ Tum8             : num  3.34 36.3 3.67 1.24 18.23 ...
 $ WT5              : num  1.14 26.84 2.88 1.05 6.45 ...
 $

### データを使いやすいように加工する

In [45]:
df2 = df1 %>% select(mice_gene_symbol,human_gene_symbol,Tum1, Tum2, Tum3, WT1, WT2)

In [46]:
df2 %<>% rename(
    macrophage_tum1 = Tum1,
    macrophage_tum2 = Tum2,
    macrophage_tum3 = Tum3,
    macrophage_wt1 = WT1,
    macrophage_wt2 = WT2
)

In [48]:
dim(df2)
df2 %>% head

[1] 16024     7

mice_gene_symbol,human_gene_symbol,macrophage_tum1,macrophage_tum2,macrophage_tum3,macrophage_wt1,macrophage_wt2
0610007P14Rik,C14orf1,9.31368,10.13680,12.26660,11.92950,11.34170
0610009D07Rik,SF3B14,25.35280,32.25990,26.61000,29.73440,37.16520
0610009O20Rik,KIAA0141,12.45760,10.17690,11.95220,10.62740,7.04763
0610010F05Rik,KIAA1841,5.84091,6.54488,6.65366,4.80074,6.06263
0610010K14Rik,C17orf49,25.63690,24.08560,20.17560,25.34280,29.14040
0610011F06Rik,C16orf13,3.48174,5.79169,3.73809,5.94575,5.98899


In [49]:
inputfile2 = "data/LR_manual_revised.txt"

dflr = read_tsv(inputfile2)

Parsed with column specification:
cols(
  From = col_character(),
  To = col_character()
)


In [50]:
dflr %>% dim
dflr %>% head

[1] 1427    2

From,To
CCK,CCKAR
GAST,CCKBR
GRP,GRPR
IL17F,IL17RA
NTN1,DSCAM
SEMA3A,PLXNA1


In [51]:
table(dflr$From %in% df1$human_gene_symbol)


FALSE  TRUE 
  201  1226 

In [52]:
table(dflr$To %in% df1$human_gene_symbol)


FALSE  TRUE 
   64  1363 

In [53]:
dfhom = read_tsv("data/HOM_MouseHumanSequence.rpt")
dim(dfhom)
head(dfhom)

Parsed with column specification:
cols(
  `HomoloGene ID` = col_integer(),
  `Common Organism Name` = col_character(),
  `NCBI Taxon ID` = col_integer(),
  Symbol = col_character(),
  `EntrezGene ID` = col_integer(),
  `Mouse MGI ID` = col_character(),
  `HGNC ID` = col_character(),
  `OMIM Gene ID` = col_integer(),
  `Genetic Location` = col_character(),
  `Genomic Coordinates (mouse: GRCm38, human: GRCh37.p10)` = col_character(),
  `Nucleotide RefSeq IDs` = col_character(),
  `Protein RefSeq IDs` = col_character(),
  `SWISS_PROT IDs` = col_character()
)


[1] 39522    13

HomoloGene ID,Common Organism Name,NCBI Taxon ID,Symbol,EntrezGene ID,Mouse MGI ID,HGNC ID,OMIM Gene ID,Genetic Location,"Genomic Coordinates (mouse: GRCm38, human: GRCh37.p10)",Nucleotide RefSeq IDs,Protein RefSeq IDs,SWISS_PROT IDs
3,"mouse, laboratory",10090,Acadm,11364,MGI:87867,NA,NA,Chr3 78.77 cM,Chr3:153922357-153944632(-),NM_007382,NP_031408,P45952
3,human,9606,ACADM,34,NA,HGNC:89,607008,Chr1 p31,Chr1:76190043-76229355(+),"NM_000016,NM_001127328","NP_000007,NP_001120800,XP_005270868,XP_005270869,XP_005270870,XP_005270871",P11310
5,"mouse, laboratory",10090,Acadvl,11370,MGI:895149,NA,NA,Chr11 42.96 cM,Chr11:70010183-70015411(-),NM_017366,NP_059062,P50544
5,human,9606,ACADVL,37,NA,HGNC:92,609575,Chr17 p13.1,Chr17:7120444-7128586(+),"NM_000018,NM_001033859,NM_001270447,NM_001270448","NP_000009,NP_001029031,NP_001257376,NP_001257377",P49748
6,"mouse, laboratory",10090,Acat1,110446,MGI:87870,NA,NA,Chr9 29.12 cM,Chr9:53580522-53610382(-),NM_144784,NP_659033,Q8QZT1
6,human,9606,ACAT1,38,NA,HGNC:93,607809,Chr11 q22.3,Chr11:107992258-108018895(+),NM_000019,NP_000010,P24752


In [97]:
hoge = dfhom %>% select(`HomoloGene ID`, Symbol, `Common Organism Name`) 
hoge[339:342,]

HomoloGene ID,Symbol,Common Organism Name
292,Smn1,"mouse, laboratory"
292,SMN1,human
292,SMN2,human
293,Snca,"mouse, laboratory"


In [103]:
dfhom2 = dfhom %>% select(`HomoloGene ID`, Symbol, `Common Organism Name`) %>%
    group_by(`HomoloGene ID`, `Common Organism Name`) %>%  mutate(id=1:n()) %>%
    spread(`Common Organism Name`, Symbol)

In [107]:
dfhom2 = rbind(dfhom, dfhom) %>% select(`HomoloGene ID`, Symbol, `Common Organism Name`) %>%
    group_by(`HomoloGene ID`, `Common Organism Name`) %>%  mutate(id=1:n()) %>%
    spread(`Common Organism Name`, Symbol)

In [108]:
dfhom2 %>% filter(`HomoloGene ID` == 292)

HomoloGene ID,id,human,"mouse, laboratory"
292,1,SMN1,Smn1
292,2,SMN2,Smn1
292,3,SMN1,NA
292,4,SMN2,NA


In [117]:
dfhom2 = full_join(
    dfhom %>% select(`HomoloGene ID`, Symbol, `Common Organism Name`) %>% 
        filter(`Common Organism Name` == "mouse, laboratory") %>%
        select(-`Common Organism Name`),
    dfhom %>% select(`HomoloGene ID`, Symbol, `Common Organism Name`) %>%
        filter(`Common Organism Name` == "human") %>%
        select(-`Common Organism Name`),
    by = "HomoloGene ID"
)
dfhom2 %>% dplyr::rename(c(mouse=Symbol.x, human=Symbol.y))

ERROR: Error: All arguments must be named


In [118]:
?dplyr::rename

In [114]:
dim(dfhom2)
head(dfhom2)
dfhom2 %>% filter(`HomoloGene ID` == 292)

[1] 22894     3

HomoloGene ID,Symbol.x,Symbol.y
3,Acadm,ACADM
5,Acadvl,ACADVL
6,Acat1,ACAT1
7,Acvr1,ACVR1
9,Sgca,SGCA
12,Adsl,ADSL


HomoloGene ID,Symbol.x,Symbol.y
292,Smn1,SMN1
292,Smn1,SMN2


In [105]:
library(reshape2)


Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths



In [106]:
 dfhom %>% select(`HomoloGene ID`, Symbol, `Common Organism Name`) %>%
        dcast(`HomoloGene ID` ~ `Common Organism Name`, value.var="Symbol") %>% head

Aggregation function missing: defaulting to length


HomoloGene ID,human,"mouse, laboratory"
3,1,1
5,1,1
6,1,1
7,1,1
9,1,1
12,1,1


In [54]:
dfdiff = read_tsv("data/CD11CB_output/gene_exp.diff")
dim(dfdiff)
head(dfdiff)

Parsed with column specification:
cols(
  test_id = col_character(),
  gene_id = col_character(),
  gene = col_character(),
  locus = col_character(),
  sample_1 = col_character(),
  sample_2 = col_character(),
  status = col_character(),
  value_1 = col_double(),
  value_2 = col_double(),
  `log2(fold_change)` = col_double(),
  test_stat = col_double(),
  p_value = col_double(),
  q_value = col_double(),
  significant = col_character()
)


[1] 30743    14

test_id,gene_id,gene,locus,sample_1,sample_2,status,value_1,value_2,log2(fold_change),test_stat,p_value,q_value,significant
0610005C13Rik,0610005C13Rik,-,chr7:45567794-45589710,q1,q2,NOTEST,0.0725212,0.0442820,-0.7116840,0.000000,1.00000,1.000000,no
0610007C21Rik,0610007C21Rik,-,chr5:31036035-31054623,q1,q2,OK,50.2687000,47.8496000,-0.0711526,-0.229939,0.71855,0.839597,no
0610007N19Rik,0610007N19Rik,-,chr15:32240567-32244662,q1,q2,NOTEST,0.2385150,0.0629525,-1.9217400,0.000000,1.00000,1.000000,no
0610007P08Rik,0610007P08Rik,-,chr13:63815319-63900301,q1,q2,OK,3.5743600,3.1124300,-0.1996430,-0.579890,0.36160,0.542589,no
0610007P14Rik,0610007P14Rik,-,chr12:85815454-85824545,q1,q2,OK,11.6356000,10.5723000,-0.1382470,-0.474073,0.54570,0.711416,no
0610007P22Rik,0610007P22Rik,-,chr17:25240169-25256364,q1,q2,OK,8.7388000,6.8487500,-0.3515960,-0.945393,0.21480,0.380104,no


In [59]:
table(dfdiff$gene_id %in% dflr$From)
table(dfdiff$gene_id %in% dflr$To)


FALSE  TRUE 
30740     3 


FALSE 
30743 

In [61]:
dfdiffsig = dfdiff %>% filter(q_value < 0.05)
dim(dfdiffsig)

[1] 4540   14

In [62]:
dfdiffsig %>% head

test_id,gene_id,gene,locus,sample_1,sample_2,status,value_1,value_2,log2(fold_change),test_stat,p_value,q_value,significant
0610010O12Rik,0610010O12Rik,-,chr18:36348623-36402978,q1,q2,OK,4.19646,8.98632,1.098560,2.23989,0.00095,0.004437260,yes
0610031J06Rik,0610031J06Rik,-,chr3:88325022-88334433,q1,q2,OK,95.85410,144.76100,0.594760,2.63080,0.00020,0.001113440,yes
0610040J01Rik,0610040J01Rik,-,chr5:63812494-63899619,q1,q2,OK,1.49871,3.08289,1.040560,2.24553,0.00420,0.016218300,yes
1100001G20Rik,1100001G20Rik,-,chr11:83746939-83752646,q1,q2,OK,301.25100,29.57580,-3.348480,-18.56810,0.00005,0.000310039,yes
1110002B05Rik,1110002B05Rik,-,chr12:54645373-54656572,q1,q2,OK,39.23150,69.53890,0.825809,3.15057,0.00005,0.000310039,yes
1110003E01Rik,1110003E01Rik,-,chr5:65448754-65492835,q1,q2,OK,20.49960,34.26780,0.741253,3.10029,0.00005,0.000310039,yes


In [63]:
table(dfdiffsig$gene_id %in% dflr$From)
table(dfdiffsig$gene_id %in% dflr$To)


FALSE  TRUE 
 4539     1 


FALSE 
 4540 

In [64]:
table(dfdiff$gene_id %in% df2$mice_gene_symbol)


FALSE  TRUE 
14743 16000 

In [65]:
dfdiffsig %<>% rename(mice_gene_symbol = gene_id)

In [71]:
dfnew = inner_join(dfdiffsig, df2, by="mice_gene_symbol")

dim(dfdiffsig)
dim(df2)
dim(dfnew)

[1] 4540   14

[1] 16024     7

[1] 3960   20

In [72]:
dfnew %>% head

test_id,mice_gene_symbol,gene,locus,sample_1,sample_2,status,value_1,value_2,log2(fold_change),test_stat,p_value,q_value,significant,human_gene_symbol,macrophage_tum1,macrophage_tum2,macrophage_tum3,macrophage_wt1,macrophage_wt2
0610031J06Rik,0610031J06Rik,-,chr3:88325022-88334433,q1,q2,OK,95.85410,144.76100,0.594760,2.63080,0.00020,0.001113440,yes,C1orf85,133.42500,127.52200,173.33500,94.28420,97.424100
0610040J01Rik,0610040J01Rik,-,chr5:63812494-63899619,q1,q2,OK,1.49871,3.08289,1.040560,2.24553,0.00420,0.016218300,yes,C4orf19,2.96581,2.68016,3.60270,2.28981,0.707607
1110004E09Rik,1110004E09Rik,-,chr16:90925810-90934849,q1,q2,OK,6.87182,3.61107,-0.928265,-2.01664,0.00230,0.009651240,yes,C21orf59,3.24060,4.02885,3.56376,6.85421,6.889420
1110007C09Rik,1110007C09Rik,-,chr13:49202950-49216026,q1,q2,OK,22.56080,77.52120,1.780770,5.93473,0.00005,0.000310039,yes,C9orf89,73.81260,82.17620,76.57470,20.49440,24.627200
1110008P14Rik,1110008P14Rik,-,chr2:32379100-32381915,q1,q2,OK,44.86820,31.28500,-0.520225,-2.38585,0.01150,0.038342300,yes,C9orf16,30.46420,31.81370,31.57710,41.20380,48.532700
1110038F14Rik,1110038F14Rik,-,chr15:76948508-76950731,q1,q2,OK,19.35170,13.43940,-0.525995,-1.59101,0.01425,0.045744300,yes,C8orf33,10.98070,14.76960,14.56780,20.45830,18.245000


In [73]:
dfnew %<>% mutate(is_receptor = human_gene_symbol %in% dflr$To)

In [74]:
dfnew %<>% mutate(is_ligand = human_gene_symbol %in% dflr$From)

In [76]:
sum(dfnew$is_ligand)
sum(dfnew$is_receptor)

[1] 100

[1] 110

differentially regulated ligands in the intratumoral stromal cells and tumor epithelial cells compared to WT counterparts, using a set of selection criteria based on FPKM > 2, fold change > 1.5, and adjusted p value < 0.1

In [80]:
deg_macrophage = read_tsv("data/CD11CB_output/gene_exp.diff")
dim(deg_macrophage)
head(deg_macrophage)

Parsed with column specification:
cols(
  test_id = col_character(),
  gene_id = col_character(),
  gene = col_character(),
  locus = col_character(),
  sample_1 = col_character(),
  sample_2 = col_character(),
  status = col_character(),
  value_1 = col_double(),
  value_2 = col_double(),
  `log2(fold_change)` = col_double(),
  test_stat = col_double(),
  p_value = col_double(),
  q_value = col_double(),
  significant = col_character()
)


[1] 30743    14

test_id,gene_id,gene,locus,sample_1,sample_2,status,value_1,value_2,log2(fold_change),test_stat,p_value,q_value,significant
0610005C13Rik,0610005C13Rik,-,chr7:45567794-45589710,q1,q2,NOTEST,0.0725212,0.0442820,-0.7116840,0.000000,1.00000,1.000000,no
0610007C21Rik,0610007C21Rik,-,chr5:31036035-31054623,q1,q2,OK,50.2687000,47.8496000,-0.0711526,-0.229939,0.71855,0.839597,no
0610007N19Rik,0610007N19Rik,-,chr15:32240567-32244662,q1,q2,NOTEST,0.2385150,0.0629525,-1.9217400,0.000000,1.00000,1.000000,no
0610007P08Rik,0610007P08Rik,-,chr13:63815319-63900301,q1,q2,OK,3.5743600,3.1124300,-0.1996430,-0.579890,0.36160,0.542589,no
0610007P14Rik,0610007P14Rik,-,chr12:85815454-85824545,q1,q2,OK,11.6356000,10.5723000,-0.1382470,-0.474073,0.54570,0.711416,no
0610007P22Rik,0610007P22Rik,-,chr17:25240169-25256364,q1,q2,OK,8.7388000,6.8487500,-0.3515960,-0.945393,0.21480,0.380104,no


In [83]:
deg_macrophage2 = left_join(deg_macrophage, dflr, by=c("gene_id" = "mice_gene_symbol"))


ERROR: Error: `by` can't contain join column `mice_gene_symbol` which is missing from RHS


In [79]:
deg_macrophage =
    dfdiff %>% filter(`log2(fold_change)` > 1.5, value_1 > 2, q_value < 0.1)
deg_macrophage_

[1] 390  14

In [ ]:
dfhom %>% gather()

In [91]:
stocks <- data.frame(
  time = as.Date('2009-01-01') + 0:9,
  X = rnorm(10, 0, 1),
  Y = rnorm(10, 0, 2),
  Z = rnorm(10, 0, 4)
)
stocksm <- stocks %>% gather(stock, price, -time)
stocksm

stocks

time,stock,price
2009-01-01,X,0.7533872
2009-01-02,X,-0.5477809
2009-01-03,X,-0.6300103
2009-01-04,X,-0.2954315
2009-01-05,X,0.6227390
2009-01-06,X,-1.7593439
2009-01-07,X,0.4836467
2009-01-08,X,-0.7243929
2009-01-09,X,-0.6973255
2009-01-10,X,-0.6319607


time,X,Y,Z
2009-01-01,0.7533872,-2.2543771,1.6562782
2009-01-02,-0.5477809,0.4906560,2.1241523
2009-01-03,-0.6300103,-2.6568473,-0.1450357
2009-01-04,-0.2954315,-0.7726514,0.3297469
2009-01-05,0.6227390,1.6084312,2.5607537
2009-01-06,-1.7593439,0.7387330,3.3061319
2009-01-07,0.4836467,-0.6197154,-2.9782445
2009-01-08,-0.7243929,0.6752242,-5.2418441
2009-01-09,-0.6973255,-1.4196808,13.2166112
2009-01-10,-0.6319607,-2.0067363,1.3511395


stock,2009-01-01,2009-01-02,2009-01-03,2009-01-04,2009-01-05,2009-01-06,2009-01-07,2009-01-08,2009-01-09,2009-01-10
X,0.7533872,-0.5477809,-0.6300103,-0.2954315,0.622739,-1.759344,0.4836467,-0.7243929,-0.6973255,-0.6319607
Y,-2.2543771,0.4906560,-2.6568473,-0.7726514,1.608431,0.738733,-0.6197154,0.6752242,-1.4196808,-2.0067363
Z,1.6562782,2.1241523,-0.1450357,0.3297469,2.560754,3.306132,-2.9782445,-5.2418441,13.2166112,1.3511395
